# NASA C-MAPSS Turbofan Engine Degradation Analysis

## Scientific Data Processing & Analysis Pipeline

This notebook implements a reproducible pipeline for analyzing turbofan engine degradation data with emphasis on:
- **Statistical validation** before machine learning
- **Multiple modeling approaches** (Linear, Neural Network, LSTM)
- **Explainability** using SHAP values
- **Scientific visualization** for clear interpretation

---

### Dataset: NASA C-MAPSS FD001
- **Source**: NASA Prognostics Center of Excellence
- **Task**: Remaining Useful Life (RUL) prediction
- **Features**: 21 sensor measurements + 3 operational settings
- **Engines**: 100 training + 100 test

---

## Table of Contents

1. [Setup & Data Loading](#1.-Setup-&-Data-Loading)
2. [Exploratory Data Analysis](#2.-Exploratory-Data-Analysis)
3. [Preprocessing & Feature Engineering](#3.-Preprocessing-&-Feature-Engineering)
4. [Statistical Analysis](#4.-Statistical-Analysis)
5. [Machine Learning Models](#5.-Machine-Learning-Models)
   - 5.1 Linear Regression Baseline
   - 5.2 Simple Neural Network
   - 5.3 LSTM for Time-Series
6. [Explainability Analysis (SHAP)](#6.-Explainability-Analysis)
7. [Model Comparison & Conclusions](#7.-Model-Comparison-&-Conclusions)

---
## 1. Setup & Data Loading

In [ ]:
# Standard imports
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.insert(0, os.path.abspath('..'))

# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

# Project modules
from src.data.ingestion import CMAPSSDataLoader, compute_training_rul, get_sensor_columns, get_feature_columns
from src.data.preprocessing import DataPreprocessor, add_degradation_features, split_by_engine

print("✅ Libraries imported successfully")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

In [ ]:
# Load the FD001 dataset
DATA_DIR = '../data/raw'

loader = CMAPSSDataLoader(DATA_DIR)
print(f"Available datasets: {loader.available_datasets}")

# Load FD001
train_df, test_df, rul_df = loader.load_dataset('FD001')

# Print summary
loader.print_summary(train_df, test_df, rul_df)

In [ ]:
# Compute RUL for training data
# Using piecewise-linear RUL with cap at 125 cycles (standard approach)
train_df = compute_training_rul(train_df, cap_rul=125)

print(f"\nRUL Statistics (Training):")
print(train_df['RUL'].describe())

In [ ]:
# Preview the data
print("\n📋 Training Data Sample:")
display(train_df.head(10))

print(f"\n📊 Data Shape: {train_df.shape}")
print(f"Memory Usage: {train_df.memory_usage(deep=True).sum() / 1e6:.2f} MB")

---
## 2. Exploratory Data Analysis

In [ ]:
# RUL Distribution
from src.visualization.plots import plot_rul_distribution

plot_rul_distribution(train_df, rul_df, save_path='../reports/figures/rul_distribution.png')

In [ ]:
# Sensor degradation trends
from src.visualization.plots import plot_sensor_degradation

# Select engines with different lifecycle lengths
lifecycle_lengths = train_df.groupby('engine_id')['cycle'].max()
sample_engines = [
    lifecycle_lengths.idxmin(),  # Shortest lifecycle
    lifecycle_lengths.idxmax(),  # Longest lifecycle
    lifecycle_lengths.median().astype(int),  # Median
]
# Add 2 random engines
remaining = [e for e in train_df['engine_id'].unique() if e not in sample_engines]
sample_engines.extend(np.random.choice(remaining, 2, replace=False).tolist())

plot_sensor_degradation(
    train_df,
    sensors=['sensor_2', 'sensor_3', 'sensor_4', 'sensor_11', 'sensor_12', 'sensor_15'],
    engine_ids=sample_engines,
    save_path='../reports/figures/sensor_degradation.png'
)

In [ ]:
# Sensor variance analysis - identify constant/uninformative sensors
sensor_cols = get_sensor_columns()
sensor_stats = train_df[sensor_cols].agg(['mean', 'std', 'min', 'max'])

print("📊 Sensor Statistics:")
display(sensor_stats.T.round(4))

In [ ]:
# Identify constant sensors (zero or near-zero variance)
variances = train_df[sensor_cols].var()
constant_sensors = variances[variances < 1e-5].index.tolist()

print(f"\n⚠️ Constant/Near-Constant Sensors (will be dropped):")
for sensor in constant_sensors:
    print(f"  • {sensor}: variance = {variances[sensor]:.2e}")

---
## 3. Preprocessing & Feature Engineering

In [ ]:
# Initialize preprocessor
preprocessor = DataPreprocessor(
    normalization='minmax',
    rolling_windows=[5, 10],  # Rolling statistics windows
    drop_constant_sensors=True
)

# Fit on training data and transform
train_processed = preprocessor.fit_transform(train_df, add_features=True)

print(f"\n✅ Preprocessing complete")
print(f"Original features: {len(get_feature_columns())}")
print(f"After preprocessing: {len([c for c in train_processed.columns if c not in ['engine_id', 'cycle', 'RUL']])}")
print(f"\nDropped sensors: {preprocessor.constant_sensors_}")

In [ ]:
# Add degradation-specific features
train_processed = add_degradation_features(train_processed)

print(f"Final feature count: {len(train_processed.columns)}")
print(f"\nNew columns added:")
new_cols = [c for c in train_processed.columns if 'diff' in c or 'ewm' in c or 'cycle_norm' in c]
print(f"  • Difference features: {len([c for c in new_cols if 'diff' in c])}")
print(f"  • EWM features: {len([c for c in new_cols if 'ewm' in c])}")
print(f"  • Normalized cycle: 1")

In [ ]:
# Split training data for validation (by engine to prevent leakage)
train_split, val_split = split_by_engine(train_processed, test_size=0.2, random_state=42)

print(f"\n📊 Train/Validation Split (by engine):")
print(f"  Training:   {len(train_split):,} samples ({train_split['engine_id'].nunique()} engines)")
print(f"  Validation: {len(val_split):,} samples ({val_split['engine_id'].nunique()} engines)")

In [ ]:
# Define feature columns for modeling (exclude metadata)
exclude_cols = ['engine_id', 'cycle', 'RUL', 'max_cycle', 'cycle_norm']
feature_cols = [c for c in train_processed.columns if c not in exclude_cols]

print(f"\n🔧 Features for modeling: {len(feature_cols)}")
print(f"\nFeature types:")
print(f"  • Base sensors: {len([c for c in feature_cols if c.startswith('sensor') and '_' not in c[7:]])}")
print(f"  • Rolling features: {len([c for c in feature_cols if 'roll' in c])}")
print(f"  • Derived features: {len([c for c in feature_cols if 'diff' in c or 'ewm' in c])}")

---
## 4. Statistical Analysis

**Important**: Statistical analysis before ML helps us understand:
- Which sensors correlate with degradation
- How distributions change as failure approaches
- Whether our data meets modeling assumptions

In [ ]:
from src.analysis.statistical import (
    compute_sensor_rul_correlation,
    analyze_sensor_monotonicity,
    compare_health_stages,
    print_statistical_summary
)

# Run comprehensive statistical summary
print_statistical_summary(train_df, rul_column='RUL')

In [ ]:
# Detailed correlation analysis
corr_df = compute_sensor_rul_correlation(train_df, method='both')

print("\n📊 Sensor-RUL Correlations:")
display(corr_df.head(15))

In [ ]:
# Visualize correlation matrix
from src.visualization.plots import plot_correlation_heatmap
from src.analysis.correlation import compute_inter_sensor_correlation

# Get active sensors (non-constant)
active_sensors = preprocessor.get_active_sensors()
corr_matrix = train_df[active_sensors].corr(method='spearman')

plot_correlation_heatmap(
    corr_matrix,
    title='Inter-Sensor Spearman Correlation',
    save_path='../reports/figures/correlation_heatmap.png'
)

In [ ]:
# Monotonicity analysis - important for prognostics
mono_df = analyze_sensor_monotonicity(train_df)

print("\n📈 Sensor Monotonicity Analysis:")
print("(Higher score = more consistent trend toward failure)\n")
display(mono_df.head(10))

In [ ]:
# Distribution shift analysis: healthy vs. degraded
stage_df = compare_health_stages(train_df, rul_threshold=50)

print("\n🔬 Healthy vs Degraded Stage Comparison:")
display(stage_df[['sensor', 'healthy_mean', 'degraded_mean', 'cohens_d', 'effect_size']].head(10))

---
## 5. Machine Learning Models

We'll train three progressively complex models:
1. **Linear Regression** - Interpretable baseline
2. **Simple Neural Network** - Non-linear relationships
3. **LSTM** - Temporal dependencies

### 5.1 Linear Regression Baseline

In [ ]:
from src.models.baseline import LinearRULPredictor, train_linear_baseline, print_linear_model_summary
from src.models.evaluation import compute_all_metrics, print_evaluation_summary

# Prepare data
X_train = train_split[feature_cols]
y_train = train_split['RUL']
X_val = val_split[feature_cols]
y_val = val_split['RUL']

print(f"Training: {X_train.shape}, Validation: {X_val.shape}")

In [ ]:
# Train Linear Regression with Ridge regularization
linear_model, cv_results = train_linear_baseline(
    train_split,
    feature_cols=feature_cols,
    regularization='ridge',
    alpha=1.0
)

print_linear_model_summary(linear_model, cv_results)

In [ ]:
# Evaluate on validation set
y_pred_linear = linear_model.predict(X_val)

print_evaluation_summary(y_val.values, y_pred_linear, 'Linear Regression')

In [ ]:
# Visualize predictions
from src.visualization.plots import plot_prediction_vs_actual, plot_residuals

plot_prediction_vs_actual(
    y_val.values, y_pred_linear,
    model_name='Linear Regression',
    save_path='../reports/figures/linear_pred_vs_actual.png'
)

In [ ]:
plot_residuals(
    y_val.values, y_pred_linear,
    model_name='Linear Regression',
    save_path='../reports/figures/linear_residuals.png'
)

### 5.2 Simple Neural Network

In [ ]:
from src.models.neural_network import SimpleNeuralNetwork, plot_training_history

# Initialize neural network
nn_model = SimpleNeuralNetwork(
    input_dim=len(feature_cols),
    hidden_dims=[128, 64, 32],
    dropout_rate=0.3,
    learning_rate=0.001
)

print(f"Device: {nn_model.device}")
print(f"\nNetwork Architecture:")
print(nn_model.model)

In [ ]:
# Train neural network
history_nn = nn_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.1,
    verbose=True
)

In [ ]:
# Plot training history
plot_training_history(
    history_nn,
    title='Neural Network Training History',
    save_path='../reports/figures/nn_training_history.png'
)

In [ ]:
# Evaluate neural network
y_pred_nn = nn_model.predict(X_val)

print_evaluation_summary(y_val.values, y_pred_nn, 'Neural Network')

In [ ]:
plot_prediction_vs_actual(
    y_val.values, y_pred_nn,
    model_name='Neural Network',
    save_path='../reports/figures/nn_pred_vs_actual.png'
)

### 5.3 LSTM for Time-Series

In [ ]:
from src.models.neural_network import LSTMModel

# Prepare sequences for LSTM
SEQUENCE_LENGTH = 30

# Use simpler feature set for LSTM (base sensors only, normalized)
base_sensor_cols = [c for c in feature_cols if 'roll' not in c and 'diff' not in c and 'ewm' not in c]

print(f"Preparing sequences with length={SEQUENCE_LENGTH}")
print(f"Using {len(base_sensor_cols)} base features")

In [ ]:
# Create sequences from training data
X_seq_train, y_seq_train = LSTMModel.prepare_sequences(
    train_split,
    feature_cols=base_sensor_cols,
    target_col='RUL',
    sequence_length=SEQUENCE_LENGTH
)

X_seq_val, y_seq_val = LSTMModel.prepare_sequences(
    val_split,
    feature_cols=base_sensor_cols,
    target_col='RUL',
    sequence_length=SEQUENCE_LENGTH
)

print(f"\nSequence shapes:")
print(f"  Training: X={X_seq_train.shape}, y={y_seq_train.shape}")
print(f"  Validation: X={X_seq_val.shape}, y={y_seq_val.shape}")

In [ ]:
# Initialize LSTM model
lstm_model = LSTMModel(
    input_dim=len(base_sensor_cols),
    hidden_dim=64,
    num_layers=2,
    dropout=0.2,
    learning_rate=0.001
)

print(f"Device: {lstm_model.device}")
print(f"\nLSTM Architecture:")
print(lstm_model.model)

In [ ]:
# Train LSTM
history_lstm = lstm_model.fit(
    X_seq_train, y_seq_train,
    epochs=50,
    batch_size=64,
    validation_split=0.1,
    verbose=True
)

In [ ]:
plot_training_history(
    history_lstm,
    title='LSTM Training History',
    save_path='../reports/figures/lstm_training_history.png'
)

In [ ]:
# Evaluate LSTM
y_pred_lstm = lstm_model.predict(X_seq_val)

print_evaluation_summary(y_seq_val, y_pred_lstm, 'LSTM')

In [ ]:
plot_prediction_vs_actual(
    y_seq_val, y_pred_lstm,
    model_name='LSTM',
    save_path='../reports/figures/lstm_pred_vs_actual.png'
)

---
## 6. Explainability Analysis

Using SHAP values to understand model predictions.

### Why Explainability Matters for Scientific Applications:
- Validates that model uses physically meaningful features
- Identifies potential data leakage or spurious correlations
- Supports trust in safety-critical decisions

In [ ]:
# For SHAP analysis, we'll use a Random Forest for better interpretability
from src.models.ensemble import RandomForestRUL, train_random_forest

# Train RF for SHAP (tree-based models have fast SHAP computation)
rf_model, rf_cv = train_random_forest(
    train_split,
    feature_cols=feature_cols,
    n_estimators=100,
    max_depth=15
)

print(f"\nRandom Forest OOB Score: {rf_model.get_oob_score():.4f}")
print(f"CV RMSE: {rf_cv['mean']:.2f} ± {rf_cv['std']:.2f}")

In [ ]:
from src.explainability.shap_analysis import SHAPExplainer, print_shap_summary

# Create SHAP explainer using Random Forest
background_sample = X_train.sample(min(200, len(X_train)), random_state=42)
shap_explainer = SHAPExplainer(rf_model, background_sample)

# Compute SHAP values on validation sample
shap_sample = X_val.sample(min(500, len(X_val)), random_state=42)
shap_values = shap_explainer.compute_shap_values(shap_sample)

print(f"\n✅ SHAP values computed for {len(shap_sample)} samples")

In [ ]:
# Print SHAP summary
print_shap_summary(shap_explainer)

In [ ]:
# SHAP Summary Plot (bar)
shap_explainer.plot_summary(
    plot_type='bar',
    max_features=15,
    save_path='../reports/figures/shap_summary_bar.png'
)

In [ ]:
# SHAP Beeswarm Plot (detailed)
shap_explainer.plot_summary(
    plot_type='beeswarm',
    max_features=15,
    save_path='../reports/figures/shap_summary_beeswarm.png'
)

In [ ]:
# Explain a single prediction
sample_idx = 0  # First validation sample
explanation = shap_explainer.explain_instance(shap_sample, index=sample_idx)

print(f"\n🔍 Single Prediction Explanation (Sample {sample_idx}):")
print(f"\nTop 10 Contributing Features:")
display(explanation.head(10))

In [ ]:
# Dependence plot for most important sensor
top_feature = shap_explainer.get_feature_importance().iloc[0]['feature']

shap_explainer.plot_dependence(
    top_feature,
    save_path=f'../reports/figures/shap_dependence_{top_feature}.png'
)

---
## 7. Model Comparison & Conclusions

In [ ]:
from src.models.evaluation import compare_models

# Collect all predictions (using same validation set size for fairness)
# Note: LSTM uses different data shape, so we compare on comparable samples

# For fair comparison, use the standard validation set
predictions = {
    'Linear Regression': y_pred_linear,
    'Neural Network': y_pred_nn,
}

# Compare models
comparison = compare_models(y_val.values, predictions)

# Add LSTM separately (different validation set)
lstm_metrics = compute_all_metrics(y_seq_val, y_pred_lstm)
lstm_row = pd.DataFrame([{
    'model': 'LSTM',
    'rmse': lstm_metrics['rmse'],
    'mae': lstm_metrics['mae'],
    'r2': lstm_metrics['r2'],
    'mape': lstm_metrics['mape'],
    'nasa_score': lstm_metrics['nasa_score']
}])

comparison = pd.concat([comparison, lstm_row], ignore_index=True)

print("\n📊 MODEL COMPARISON:")
print("=" * 70)
display(comparison)

In [ ]:
# Visualization of model comparison
from src.visualization.plots import plot_model_comparison

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE comparison
colors = plt.cm.Set2(np.linspace(0, 1, len(comparison)))
axes[0].bar(comparison['model'], comparison['rmse'], color=colors)
axes[0].set_ylabel('RMSE (cycles)')
axes[0].set_title('RMSE Comparison (Lower is Better)')
for i, v in enumerate(comparison['rmse']):
    axes[0].text(i, v + 0.5, f'{v:.2f}', ha='center', fontsize=11)

# R² comparison
axes[1].bar(comparison['model'], comparison['r2'], color=colors)
axes[1].set_ylabel('R² Score')
axes[1].set_title('R² Comparison (Higher is Better)')
for i, v in enumerate(comparison['r2']):
    axes[1].text(i, v + 0.01, f'{v:.3f}', ha='center', fontsize=11)

plt.tight_layout()
plt.savefig('../reports/figures/model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Create summary dashboard
from src.visualization.plots import create_summary_dashboard

# Use best model for dashboard (typically Neural Network or LSTM)
best_model_idx = comparison['rmse'].idxmin()
best_model_name = comparison.loc[best_model_idx, 'model']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"  RMSE: {comparison.loc[best_model_idx, 'rmse']:.2f} cycles")
print(f"  MAE: {comparison.loc[best_model_idx, 'mae']:.2f} cycles")
print(f"  R²: {comparison.loc[best_model_idx, 'r2']:.4f}")

---
## Summary & Key Findings

### Statistical Insights:
1. **Constant sensors** (1, 5, 6, 10, 16, 18, 19) provide no predictive value
2. **Sensors 4, 11, 12, 15** show strongest correlation with RUL
3. Clear distribution shifts between healthy (RUL > 50) and degraded states

### Model Performance:
- **Linear Regression**: Provides interpretable baseline
- **Neural Network**: Captures non-linear patterns
- **LSTM**: Leverages temporal dependencies (best for sequential prediction)

### Explainability (SHAP):
- Model correctly uses physically meaningful sensors
- No obvious signs of data leakage
- Feature importance aligns with domain knowledge

---

### Future Work:
1. Extend to FD002-FD004 (multiple operating conditions)
2. Implement attention mechanisms for LSTM
3. Add uncertainty quantification
4. Deploy as RESTful API for predictions

In [ ]:
# Save final comparison results
comparison.to_csv('../reports/model_comparison.csv', index=False)
print("\n✅ Results saved to reports/model_comparison.csv")